In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!nvidia-smi

Mon Apr 18 12:44:05 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
!ln -s /content/drive/MyDrive/ /gdrive

In [5]:
!cp /gdrive/ETRI/path_data/path_train.pkl ./
!cp /gdrive/ETRI/path_data/path_dev.pkl ./
!cp /gdrive/ETRI/path_data/path_test.pkl ./
!cp /gdrive/ETRI/KEMDy19.zip ./

In [6]:
!unzip KEMDy19.zip

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: KEMDy19/wav/Session17/Sess17_script04/Sess17_script04_M011.wav  
  inflating: KEMDy19/wav/Session17/Sess17_script04/Sess17_script04_M012.csv  
  inflating: KEMDy19/wav/Session17/Sess17_script04/Sess17_script04_M012.txt  
  inflating: KEMDy19/wav/Session17/Sess17_script04/Sess17_script04_M012.wav  
  inflating: KEMDy19/wav/Session17/Sess17_script04/Sess17_script04_M013.csv  
 extracting: KEMDy19/wav/Session17/Sess17_script04/Sess17_script04_M013.txt  
  inflating: KEMDy19/wav/Session17/Sess17_script04/Sess17_script04_M013.wav  
  inflating: KEMDy19/wav/Session17/Sess17_script04/Sess17_script04_M014.csv  
  inflating: KEMDy19/wav/Session17/Sess17_script04/Sess17_script04_M014.txt  
  inflating: KEMDy19/wav/Session17/Sess17_script04/Sess17_script04_M014.wav  
  inflating: KEMDy19/wav/Session17/Sess17_script04/Sess17_script04_M015.csv  
  inflating: KEMDy19/wav/Session17/Sess17_script04/Sess17_script04_M015.txt  
  inflating: KEMDy19/wav/Ses

In [7]:
!pip install transformers datasets librosa

     |████████████████████████████████| 4.0 MB 8.8 MB/s 
     |████████████████████████████████| 325 kB 62.4 MB/s 
     |████████████████████████████████| 77 kB 8.6 MB/s 
     |████████████████████████████████| 596 kB 77.7 MB/s 
     |████████████████████████████████| 895 kB 69.8 MB/s 
     |████████████████████████████████| 6.6 MB 59.8 MB/s 
     |████████████████████████████████| 136 kB 77.0 MB/s 
     |████████████████████████████████| 1.1 MB 72.3 MB/s 
     |████████████████████████████████| 212 kB 81.4 MB/s 
     |████████████████████████████████| 127 kB 85.8 MB/s 
     |████████████████████████████████| 94 kB 4.3 MB/s 
     |████████████████████████████████| 271 kB 75.1 MB/s 
     |████████████████████████████████| 144 kB 77.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling

In [8]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import sys
import warnings
import re

import numpy as np
import pandas as pd
import librosa

import datasets
from datasets import load_metric
from transformers import HubertForSequenceClassification, Wav2Vec2FeatureExtractor, TrainingArguments, Trainer

import torch
from torch.utils.data import Dataset, DataLoader

warnings.filterwarnings(action='ignore')

In [9]:
audio_model = HubertForSequenceClassification.from_pretrained("/gdrive/ETRI/backup/hubert_large/checkpoint-4110", num_labels=7)
audio_feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("/gdrive/ETRI/backup/hubert_large/checkpoint-4110")

In [10]:
audio_model

HubertForSequenceClassification(
  (hubert): HubertModel(
    (feature_extractor): HubertFeatureEncoder(
      (conv_layers): ModuleList(
        (0): HubertLayerNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (1): HubertLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (2): HubertLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (3): HubertLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, ele

In [11]:
class AudioTextDataset(Dataset):
    
    def __init__(self, 
                 data_path: str):
        super(AudioTextDataset, self).__init__()
        self.data_path = data_path
        self.audio_path, self.labels = self.load_data(self.data_path)
        self.feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("/gdrive/ETRI/backup/hubert_large/checkpoint-4110")
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        audio = librosa.load(self.audio_path[idx], sr=16000)[0]
        audio_input = self.tokenizer(audio)
        item = {key: torch.tensor(val).squeeze() for key, val in audio_input.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
  

    @staticmethod
    def load_data(data_path: str):
        data = pd.read_pickle(data_path)
        data = data.dropna()
        audio_path = data['total_path'].values
        labels = data['emotion'].values
        return audio_path, labels

    def tokenizer(self, audio_path):
        feature_extractor = self.feature_extractor
        input = feature_extractor(audio_path, sampling_rate=16000, max_length=80000, padding="max_length", truncation=True, return_tensors="pt")
        return input

In [12]:
data_train = AudioTextDataset('path_train.pkl')
data_val = AudioTextDataset('path_dev.pkl')
data_test = AudioTextDataset('path_test.pkl')

In [13]:
metric = load_metric("glue", "mnli")

In [14]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [15]:
def model_init():
    return HubertForSequenceClassification.from_pretrained("/gdrive/ETRI/backup/hubert_large/checkpoint-4110", num_labels=7, output_hidden_states=False)

In [16]:
metric_name = "accuracy"

args = TrainingArguments(
    "saved",
    evaluation_strategy="epoch",
    save_strategy='epoch',
    learning_rate=2e-05,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    greater_is_better=True,
    seed = 42
)

In [17]:
trainer = Trainer(
    model_init=model_init,
    args=args,
    train_dataset=data_train,
    eval_dataset=data_val,
    tokenizer=audio_feature_extractor,
    compute_metrics=compute_metrics
)

loading configuration file /gdrive/ETRI/backup/hubert_large/checkpoint-4110/config.json
Model config HubertConfig {
  "_name_or_path": "facebook/hubert-large-ls960-ft",
  "activation_dropout": 0.1,
  "apply_spec_augment": true,
  "architectures": [
    "HubertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 1,
  "classifier_proj_size": 256,
  "conv_bias": true,
  "conv_dim": [
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "conv_kernel": [
    10,
    3,
    3,
    3,
    3,
    2,
    2
  ],
  "conv_stride": [
    5,
    2,
    2,
    2,
    2,
    2,
    2
  ],
  "ctc_loss_reduction": "sum",
  "ctc_zero_infinity": false,
  "diversity_loss_weight": 0.1,
  "do_stable_layer_norm": true,
  "eos_token_id": 2,
  "feat_extract_activation": "gelu",
  "feat_extract_dropout": 0.0,
  "feat_extract_norm": "layer",
  "feat_proj_dropout": 0.1,
  "feat_proj_layer_norm": true,
  "final_dropout": 0.1,
  "gradient_checkpointing": false,
  "hidden_act"

In [18]:
trainer.train()

loading configuration file /gdrive/ETRI/backup/hubert_large/checkpoint-4110/config.json
Model config HubertConfig {
  "_name_or_path": "facebook/hubert-large-ls960-ft",
  "activation_dropout": 0.1,
  "apply_spec_augment": true,
  "architectures": [
    "HubertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 1,
  "classifier_proj_size": 256,
  "conv_bias": true,
  "conv_dim": [
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "conv_kernel": [
    10,
    3,
    3,
    3,
    3,
    2,
    2
  ],
  "conv_stride": [
    5,
    2,
    2,
    2,
    2,
    2,
    2
  ],
  "ctc_loss_reduction": "sum",
  "ctc_zero_infinity": false,
  "diversity_loss_weight": 0.1,
  "do_stable_layer_norm": true,
  "eos_token_id": 2,
  "feat_extract_activation": "gelu",
  "feat_extract_dropout": 0.0,
  "feat_extract_norm": "layer",
  "feat_proj_dropout": 0.1,
  "feat_proj_layer_norm": true,
  "final_dropout": 0.1,
  "gradient_checkpointing": false,
  "hidden_act"

Epoch,Training Loss,Validation Loss,Accuracy
1,1.193800,1.304341,0.574209
2,1.107400,1.316432,0.568735
3,1.024700,1.207225,0.607664
4,0.927900,1.260813,0.591849
5,0.891400,1.279982,0.589416


***** Running Evaluation *****
  Num examples = 1644
  Batch size = 8
Saving model checkpoint to saved/checkpoint-822
Configuration saved in saved/checkpoint-822/config.json
Model weights saved in saved/checkpoint-822/pytorch_model.bin
Feature extractor saved in saved/checkpoint-822/preprocessor_config.json
***** Running Evaluation *****
  Num examples = 1644
  Batch size = 8
Saving model checkpoint to saved/checkpoint-1644
Configuration saved in saved/checkpoint-1644/config.json
Model weights saved in saved/checkpoint-1644/pytorch_model.bin
Feature extractor saved in saved/checkpoint-1644/preprocessor_config.json
***** Running Evaluation *****
  Num examples = 1644
  Batch size = 8
Saving model checkpoint to saved/checkpoint-2466
Configuration saved in saved/checkpoint-2466/config.json
Model weights saved in saved/checkpoint-2466/pytorch_model.bin
Feature extractor saved in saved/checkpoint-2466/preprocessor_config.json
***** Running Evaluation *****
  Num examples = 1644
  Batch size

TrainOutput(global_step=4110, training_loss=1.0226536678861817, metrics={'train_runtime': 4985.0024, 'train_samples_per_second': 6.595, 'train_steps_per_second': 0.824, 'total_flos': 4.98179206182e+18, 'train_loss': 1.0226536678861817, 'epoch': 5.0})

In [19]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 1644
  Batch size = 8


{'epoch': 5.0,
 'eval_accuracy': 0.6076642335766423,
 'eval_loss': 1.2072248458862305,
 'eval_runtime': 84.9487,
 'eval_samples_per_second': 19.353,
 'eval_steps_per_second': 2.425}

In [20]:
pred = trainer.predict(data_test)

***** Running Prediction *****
  Num examples = 2055
  Batch size = 8


In [21]:
pred

PredictionOutput(predictions=array([[ 0.6483508 ,  2.2925544 , -0.89196354, ..., -1.4931359 ,
         0.00449045, -1.0322369 ],
       [ 2.201752  ,  1.3335475 ,  0.24619296, ..., -2.9316359 ,
        -2.3818552 , -0.6956245 ],
       [ 0.8193887 , -2.0171127 , -0.06881657, ..., -0.253286  ,
         3.2720153 , -0.65734977],
       ...,
       [ 3.8763804 , -0.9645021 , -1.749038  , ..., -3.7062917 ,
         0.5947992 ,  0.5883068 ],
       [ 1.1513082 , -0.99651164, -0.77991   , ..., -2.6484141 ,
        -1.7162735 ,  0.6934089 ],
       [ 1.7758313 , -0.34546408,  1.1930712 , ..., -2.106436  ,
        -2.1418114 , -0.49978116]], dtype=float32), label_ids=array([1, 0, 5, ..., 0, 3, 2]), metrics={'test_loss': 1.1868393421173096, 'test_accuracy': 0.6155717761557178, 'test_runtime': 111.2104, 'test_samples_per_second': 18.478, 'test_steps_per_second': 2.311})

In [22]:
pred = pred[0]
pred

array([[ 0.6483508 ,  2.2925544 , -0.89196354, ..., -1.4931359 ,
         0.00449045, -1.0322369 ],
       [ 2.201752  ,  1.3335475 ,  0.24619296, ..., -2.9316359 ,
        -2.3818552 , -0.6956245 ],
       [ 0.8193887 , -2.0171127 , -0.06881657, ..., -0.253286  ,
         3.2720153 , -0.65734977],
       ...,
       [ 3.8763804 , -0.9645021 , -1.749038  , ..., -3.7062917 ,
         0.5947992 ,  0.5883068 ],
       [ 1.1513082 , -0.99651164, -0.77991   , ..., -2.6484141 ,
        -1.7162735 ,  0.6934089 ],
       [ 1.7758313 , -0.34546408,  1.1930712 , ..., -2.106436  ,
        -2.1418114 , -0.49978116]], dtype=float32)

In [23]:
np.save("/gdrive/ETRI/npy/pred_hubert_large_whole_1.npy", pred)

In [24]:
!cp -r ./saved/checkpoint-2466 /gdrive/ETRI/backup/hubert_large

In [25]:
torch.cuda.empty_cache()